In [66]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [67]:
import pandas as pd
from num2words import num2words
import re
from ast import literal_eval
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)

from sklearn.model_selection import train_test_split

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#### Собираем все воедино, удаляем лишние колонки

In [68]:
wikiann = '/home/sergey/Python_projects/RU_NER/Project_Data/Data/Wikiann_ru/Wikiann_augmented.csv'
wikineural = '/home/sergey/Python_projects/RU_NER/Project_Data/Data/Babelscape_Wikineural/Wikineural_augmented.csv'
multinerd = '/home/sergey/Python_projects/RU_NER/Project_Data/Data/MultiNerd/MultiNERD_augmented.csv'
common_voice = '/home/sergey/Python_projects/RU_NER/Project_Data/Data/CommonVoice/Edited_NER_anotation_Common_Voice.csv'

In [69]:
markup = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [70]:
df_wa = pd.read_csv(wikiann)
df_wiki = pd.read_csv(wikineural)
df_multinerd = pd.read_csv(multinerd)

In [71]:
df = pd.concat([df_wa, df_wiki, df_multinerd], ignore_index=True)

In [72]:
df

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"['Илизаров', ',', 'Гавриил', 'Абрамович']","[1, 2, 2, 2]",ok,"['илизаров', 'гавриил', 'абрамович']","[1, 2, 2]",ok
1,"[""'"", ""''"", 'Рыбницкий', 'район', ""''"", ""'""]","[0, 0, 5, 6, 0, 0]",ok,"['рыбницкий', 'район']","[5, 6]",ok
2,"['За', 'образцовое', 'выполнение', 'заданий', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ...",ok,"['за', 'образцовое', 'выполнение', 'заданий', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ...",ok
3,"['Стадион', 'имени', 'С.', 'Дарюса', 'и', 'С.'...","[5, 6, 6, 6, 6, 6, 6]",ok,"['стадион', 'имени', 'с', 'дарюса', 'и', 'с', ...","[5, 6, 6, 6, 6, 6, 6]",ok
4,"['Майкл', 'Томас', '(', '1987—1991', ')']","[1, 2, 0, 0, 0]",ok,"['майкл', 'томас', 'одна', 'тысяча', 'девятьсо...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ok
...,...,...,...,...,...,...
238239,"['Дебютировал', 'в', 'матче', '1', 'тура', 'пр...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"['дебютировал', 'в', 'матче', 'один', 'тура', ...","[0, 0, 0, 0, 0, 0, 4]",ok
238240,"['Дебют', 'состоялся', '26', 'туре', 'против',...","[0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"['дебют', 'состоялся', 'двадцать', 'шесть', 'т...","[0, 0, 0, 0, 0, 0, 4]",ok
238241,"['Эктор', 'также', 'представлял', 'свою', 'стр...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 8, 8, 0]",ok,"['эктор', 'также', 'представлял', 'свою', 'стр...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 7, 8, 8, ...",ok
238242,"['Q', 'дал', 'альбому', 'четыре', 'звезды', 'и...","[7, 0, 0, 0, 0, 0, 0, 0]",ok,"['q', 'дал', 'альбому', 'четыре', 'звезды', 'и...","[7, 0, 0, 0, 0, 0, 0]",ok


In [73]:
df['augmented_ner_tags'] = df['augmented_ner_tags'].parallel_apply(literal_eval)
df['augmented_tokens'] = df['augmented_tokens'].parallel_apply(literal_eval)

In [74]:
df.drop(['tokens', 'ner_tags', 'len_check', 'augmented_length_check'], axis=1, inplace=True)

In [75]:
df.head()

,augmented_tokens,augmented_ner_tags
0,"[илизаров, гавриил, абрамович]","[1, 2, 2]"
1,"[рыбницкий, район]","[5, 6]"
2,"[за, образцовое, выполнение, заданий, командов...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ..."
3,"[стадион, имени, с, дарюса, и, с, гиренаса]","[5, 6, 6, 6, 6, 6, 6]"
4,"[майкл, томас, одна, тысяча, девятьсот, восемь...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [76]:
df_cv = pd.read_csv(common_voice)

In [77]:
df_cv.drop(['id', 'sentence', 'difference', 'label', 'NER', 'len'], axis=1, inplace=True)

In [78]:
df_cv.columns = ['augmented_tokens', 'augmented_ner_tags']

In [79]:
df_cv['augmented_tokens'] = df_cv.augmented_tokens.apply(lambda x: x.split())

In [80]:
df_cv['augmented_ner_tags'] = df_cv['augmented_ner_tags'].parallel_apply(literal_eval)

In [81]:
df = pd.concat([df, df_cv], ignore_index=True)

In [82]:
df

,augmented_tokens,augmented_ner_tags
0,"[илизаров, гавриил, абрамович]","[1, 2, 2]"
1,"[рыбницкий, район]","[5, 6]"
2,"[за, образцовое, выполнение, заданий, командов...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ..."
3,"[стадион, имени, с, дарюса, и, с, гиренаса]","[5, 6, 6, 6, 6, 6, 6]"
4,"[майкл, томас, одна, тысяча, девятьсот, восемь...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
274300,"[анна, продолжала, идти, по, знакомой, лестниц...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
274301,"[ей, даже, неприятно, было, подумать, что, она...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
274302,"[мне, кажется, вам, уже, тяжело, дышать]","[0, 0, 0, 0, 0, 0]"
274303,"[число, совершаемых, изнасилований, достигла, ...","[0, 0, 0, 0, 0, 0, 0]"


Проверка мусора:

In [94]:
trash_index = []
for i in range(len(df)):
    if len(df.augmented_tokens[i]) == 1:
        if len(df.augmented_tokens[i][0]) <3:
            print(df.augmented_tokens[i][0])
            trash_index.append(i)
            print(i)

u2
1693
de
5687
у
11006
н
13107
u2
13925
аз
36236
mu
45591
аа
49286
и0
52255
pz
79270
nc
91150
св
98193
уз
100539
iv
144227
ну
239414
да
241323
да
241454
я
253098
да
261436
эй
261553
ах
268400
да
269391
эй
271168


In [96]:
df.iloc[241454]

augmented_tokens      [да]
augmented_ner_tags     [0]
Name: 241454, dtype: object

In [97]:
for ind in trash_index:
    df.drop(ind, inplace=True)

In [99]:
df.reset_index(drop=True, inplace=True)

#### небольшая финальная корректировка, меняем римские цифры на обычные, меняем разметку

In [100]:
roman_map = {'i': 1,'v': 5,'x': 10,'l': 50,'c': 100,'d': 500,'m': 1000}

In [101]:
def length_check(tokens, ner_tags):
    return 'ok' if len(tokens) == len(ner_tags) else 'WRONG'

In [102]:
def is_roman_numeral(token):
    roman_numeral_pattern = r"^(?=[mdclxvi])M*(c[md]|d?c{0,3})(x[cl]|l?x{0,3})(i[xv]|v?i{0,3})$"
    return bool(re.match(roman_numeral_pattern, token, re.IGNORECASE))

In [103]:
def roman_to_arabic(tokens):
    for index, token in enumerate(tokens):
        if is_roman_numeral(token):
            total = 0
            prev_value = 0
            for numeral in reversed(token):
                current_value = roman_map[numeral]
                if current_value >= prev_value:
                    total += current_value
                else:
                    total -= current_value
                prev_value = current_value
            tokens[index] = str(total)
    return tokens

В датасете есть один кривой токен, просто удалим его

In [104]:
for i in range(len(df)):
    try:
        roman_to_arabic(df.augmented_tokens[i])
    except:
        print(i)
        break

67718


In [105]:
trash = 'lı'
df.augmented_tokens[67718].index(trash)

11

In [107]:
del df['augmented_tokens'][67718][11]
del df['augmented_ner_tags'][67718][11]

In [108]:
df.reset_index(drop=True, inplace=True)

In [109]:
df['augmented_tokens'] = df['augmented_tokens'].parallel_apply(lambda x: roman_to_arabic(x))

Сделаем числовые значения текстовыми, и скорректируем нер разметку

In [110]:
def replace_numerical_with_words(tokens, ner_tags):
    adjusted_ner_tags = []
    
    for i, token in enumerate(tokens):
        if token.isnumeric():
            index = i
            new_tokens = num2words(int(token), lang='ru').split()
            tag_to_insert = ner_tags[i]
            count = len(new_tokens)
            del ner_tags[i]
            del tokens[i] 
            for _ in range(len(new_tokens)):
                ner_tags.insert(index, tag_to_insert)
                tokens.insert(index, new_tokens[_])
                index += 1 
                
    return tokens, ner_tags

In [111]:
df[['augmented_tokens', 'augmented_ner_tags']] = df.parallel_apply(lambda row: pd.Series(replace_numerical_with_words(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

Проверим что ничего не сбилось:

In [112]:
df['augmented_length_check'] = df.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df['augmented_length_check'].value_counts()

ok    274282
Name: augmented_length_check, dtype: int64

Добавим предложения из токенов, поскольку токенизация у моделей может быть разная:

In [113]:
def convert_to_sentence(lst_str):
    #lst = literal_eval(lst_str)
    return ' '.join(lst_str)

In [114]:
df['sentence'] = df['augmented_tokens'].parallel_apply(lambda x: convert_to_sentence(x))

In [115]:
df.drop('augmented_length_check', axis=1, inplace=True)

In [116]:
df = df[['augmented_tokens', 'sentence', 'augmented_ner_tags']]

In [117]:
def print_tags_frequency(tags):
    freq = {}
    for row in tags:
        for tag in set(row):
            if tag in freq:
                freq[tag] += 1
            else:
                freq[tag] = 1
    freq
    for item in sorted(freq.keys()):
        print(f'% of rows token {item} appears in is {freq[item]/len(tags)*100:.4f}')

In [118]:
print_tags_frequency(df['augmented_ner_tags'])

% of rows token 0 appears in is 94.0186
% of rows token 1 appears in is 26.3517
% of rows token 2 appears in is 17.2450
% of rows token 3 appears in is 15.6084
% of rows token 4 appears in is 9.1858
% of rows token 5 appears in is 37.2489
% of rows token 6 appears in is 7.5393
% of rows token 7 appears in is 16.3839
% of rows token 8 appears in is 7.8481


In [119]:
y = df['augmented_ner_tags']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=12)

In [121]:
print_tags_frequency(X_train['augmented_ner_tags'])

% of rows token 0 appears in is 94.0285
% of rows token 1 appears in is 26.2679
% of rows token 2 appears in is 17.2475
% of rows token 3 appears in is 15.6355
% of rows token 4 appears in is 9.2154
% of rows token 5 appears in is 37.2980
% of rows token 6 appears in is 7.5207
% of rows token 7 appears in is 16.3506
% of rows token 8 appears in is 7.8309


In [122]:
print_tags_frequency(X_test['augmented_ner_tags'])

% of rows token 0 appears in is 93.9887
% of rows token 1 appears in is 26.6031
% of rows token 2 appears in is 17.2376
% of rows token 3 appears in is 15.5270
% of rows token 4 appears in is 9.0971
% of rows token 5 appears in is 37.1017
% of rows token 6 appears in is 7.5950
% of rows token 7 appears in is 16.4836
% of rows token 8 appears in is 7.8998


In [123]:
X_train

,augmented_tokens,sentence,augmented_ner_tags
110881,"[последний, был, разбит, при, сауле, жемайтами...",последний был разбит при сауле жемайтами и зем...,"[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
168081,"[научным, руководителем, был, в, и, арнольд]",научным руководителем был в и арнольд,"[0, 0, 0, 1, 2, 2]"
266670,"[но, как, же, нынче, выдают, замуж, княгиня, н...",но как же нынче выдают замуж княгиня ни от ког...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
41382,"[в, конце, финального, эпизода, сериала, дверь...",в конце финального эпизода сериала дверь посре...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
65106,"[в, начале, олигоцена, произошло, похолодание,...",в начале олигоцена произошло похолодание клима...,"[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
206083,"[саундтрек, игры, номинировался, на, лучшую, о...",саундтрек игры номинировался на лучшую общую м...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]"
36482,"[сараево, босния, и, герцеговина]",сараево босния и герцеговина,"[5, 5, 6, 6]"
40177,"[до, одна, тысяча, девятьсот, пятьдесят, два, ...",до одна тысяча девятьсот пятьдесят два года ра...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]"
255366,"[однако, путь, к, этой, цели, предстоит, долги...",однако путь к этой цели предстоит долгий и нам...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [124]:
X_test

,augmented_tokens,sentence,augmented_ner_tags
96657,"[сын, помещика, ротмистра, в, отставке, владим...",сын помещика ротмистра в отставке владимира ви...,"[0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2]"
182493,"[в, англии, разные, нормандские, короли, царст...",в англии разные нормандские короли царствовавш...,"[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
122924,"[так, как, в, этом, павсанию, было, отказано, ...",так как в этом павсанию было отказано то он ос...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98842,"[в, дистрибутив, входит, компилятор, mingw]",в дистрибутив входит компилятор mingw,"[0, 0, 0, 0, 7]"
131859,"[приз, образован, накануне, сезона2004, когда,...",приз образован накануне сезона2004 когда закры...,"[0, 0, 0, 0, 0, 0, 0, 3, 0, 5]"
...,...,...,...
228407,"[к, тому, же, должность, главнокомандующего, ф...",к тому же должность главнокомандующего фронтом...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
187999,"[например, в, бассейне, реки, ангара, произрас...",например в бассейне реки ангара произрастает а...,"[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0]"
250994,"[основой, наших, действий, является, стремлени...",основой наших действий является стремление доб...,"[0, 0, 0, 0, 0, 0, 0, 0]"
271112,"[самый, центр, не, требует, никаких, усилий]",самый центр не требует никаких усилий,"[0, 0, 0, 0, 0, 0]"


In [125]:
X_train.to_csv('train_data.csv', index=False)
X_test.to_csv('test_data.csv', index=False)